## Data Wrangling: Clean, Transform, Merge, Reshape

In [2]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas
import pandas as pd
np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100

In [3]:
%matplotlib inline

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Combining and merging data sets

### Database-style DataFrame merges

In [5]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [6]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [7]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [8]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [9]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
df3
df4

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


,data2,rkey
0,0,a
1,1,b
2,2,d


In [10]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [11]:
df1
df2
pd.merge(df1, df2, how='outer')

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


,data2,key
0,0,a
1,1,b
2,2,d


,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [12]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})
df1
df2

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [13]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [14]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [15]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
left
right
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [16]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [17]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on index

In [18]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
left1
right1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


In [19]:
pd.merge(left1, right1, left_on='key', right_index=True)
pd.merge(right1, left1, right_on='key', left_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


,group_val,key,value
0,3.5,a,0
2,3.5,a,2
3,3.5,a,3
1,7.0,b,1
4,7.0,b,4


In [20]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [21]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5.)})
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])
lefth
righth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [22]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [23]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


In [24]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                 columns=['Ohio', 'Nevada'])
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])
left2
right2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [25]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [26]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [27]:
left1
right1
left1.join(right1, on='key')

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [28]:
left2
right2
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])
another

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [29]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [30]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


In [31]:
left2
right2
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], columns=['New York', 'Ohio'])
another

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


,New York,Ohio
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [32]:
left2.join([right2, another])

ValueError: Indexes have overlapping values: ['Ohio']

### Concatenating along an axis

In [33]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [34]:
np.concatenate([arr, arr], axis=1)
np.concatenate([arr, arr])

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [35]:
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])
s1
s2
s3

a    0
b    1
dtype: int64

c    2
d    3
e    4
dtype: int64

f    5
g    6
dtype: int64

In [36]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [37]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [38]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [39]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [40]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [41]:
s1
s4
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e', 'wow']])

a    0
b    1
dtype: int64

a    0
b    5
f    5
g    6
dtype: int64

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN
wow,NaN,NaN


In [53]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
            # concat의 keys   를 주게 되면

In [56]:
result # 계층 색인이 됨. 

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [58]:
# Much more on the unstack function later
result.unstack()  # 아래서 위로 올림  - 데이터 프레임으로 나오게 됨.

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [59]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])
        # 시리즈 형태

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [61]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                columns=['three', 'four'])
df1
df2      # 데이터 프레임을 리스트에 넣어서 이어 붙임!
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

,one,two
a,0,1
b,2,3
c,4,5


,three,four
a,5,6
c,7,8


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [71]:
pd.concat([df1,df2])
pd.concat([df1,df2], ignore_index=True)       # 인덱스 무시하고 무조건 정수 인덱스가 나옴   -  0부터 ~~~

,a,b,c,d
0,1.669025,-0.438570,-0.539741,0.476985
1,3.248944,-1.021228,-0.577087,0.124121
2,0.302614,0.523772,0.000940,1.343810
0,-2.370232,-0.713544,NaN,-0.831154
1,0.560145,-1.860761,NaN,-0.860757


,a,b,c,d
0,1.669025,-0.438570,-0.539741,0.476985
1,3.248944,-1.021228,-0.577087,0.124121
2,0.302614,0.523772,0.000940,1.343810
3,-2.370232,-0.713544,NaN,-0.831154
4,0.560145,-1.860761,NaN,-0.860757


In [62]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)
             # 사전으로 값을 줌.o

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [64]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])   # 컬럼 레벨 별로 이름을 주겠다.

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [66]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1
df2

,a,b,c,d
0,1.669025,-0.438570,-0.539741,0.476985
1,3.248944,-1.021228,-0.577087,0.124121
2,0.302614,0.523772,0.000940,1.343810


,b,d,a
0,-0.713544,-0.831154,-2.370232
1,-1.860761,-0.860757,0.560145


In [67]:
pd.concat([df1, df2])    # axis = 0 자동 /   로우 단위로 붙였는데, 
# 0,1,2,3 순서대로 보고 싶다.

,a,b,c,d
0,1.669025,-0.438570,-0.539741,0.476985
1,3.248944,-1.021228,-0.577087,0.124121
2,0.302614,0.523772,0.000940,1.343810
0,-2.370232,-0.713544,NaN,-0.831154
1,0.560145,-1.860761,NaN,-0.860757


In [68]:
pd.concat([df1, df2], ignore_index=True)   # 기존 것 무시하고, 새롭게 인덱스를 표시해줌.

,a,b,c,d
0,1.669025,-0.438570,-0.539741,0.476985
1,3.248944,-1.021228,-0.577087,0.124121
2,0.302614,0.523772,0.000940,1.343810
3,-2.370232,-0.713544,NaN,-0.831154
4,0.560145,-1.860761,NaN,-0.860757


### Combining data with overlap

In [72]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan  # b 시리즈의 마지막 값(-1)을 null 값으로 바꾸겠다.
a
b

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [79]:
# 시리즈 = merge 안됨.

np.where(pd.isnull(a), b, a)
           # 이 값이 트루이면  b를 리턴 / 아니면 a 리턴


array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [77]:
b[:-2].combine_first(a[2:])
# b - f e d c 값을 대상으로 combine   => b값이 없음. 한쪽만 null 이므로 다른 값이 나옴.
# a - f e d c 값 
# f는 제로가 나옴



a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [81]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})

df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

# 우선순위는 df1 이 우선이고, df1 값이 null 이면 df2 값이 도출됨.

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## Reshaping and pivoting

### Reshaping with hierarchical indexing

In [82]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [83]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [84]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [86]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [87]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [88]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
s1
s2
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

a    0
b    1
c    2
d    3
dtype: int64

c    4
d    5
e    6
dtype: int64

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [89]:
data2.unstack() # 아래서 위로 올려버림 (a, b,c , d 를 위로 )
data2.unstack(0)

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


,one,two
a,0.0,NaN
b,1.0,NaN
c,2.0,4.0
d,3.0,5.0
e,NaN,6.0


In [90]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [91]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [92]:
result
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))
df

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [93]:
df.unstack('state')  # state 가 칼럼으로 가고, number가 로우 인덱스로 남게됨.

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [96]:
df.unstack('state').stack('side')   # 위로 보낼 것, 아래로 보낼 것을 정하면 됨.

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting "long" to "wide" format

In [97]:
Pivoting

NameError: name 'Pivoting' is not defined

In [98]:
data = pd.read_csv('ch07/macrodata.csv')
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
5,1960.0,2.0,2834.390,1792.9,298.152,460.400,1966.1,29.550,140.2,2.68,5.2,180.671,0.14,2.55
6,1960.0,3.0,2839.022,1785.8,296.375,474.676,1967.8,29.750,140.9,2.36,5.6,181.528,2.70,-0.34
7,1960.0,4.0,2802.616,1788.2,259.764,476.434,1966.6,29.840,141.1,2.29,6.3,182.287,1.21,1.08
8,1961.0,1.0,2819.264,1787.7,266.405,475.854,1984.5,29.810,142.1,2.37,6.8,182.992,-0.40,2.77
9,1961.0,2.0,2872.005,1814.3,286.246,480.328,2014.4,29.920,142.9,2.29,7.0,183.691,1.47,0.81


In [100]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods                       # data의 year 항목을 year로 
               # 시계열 데이터를 위한 인덱스

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [101]:
data.to_records()      # 원래 데이터는 dataframe 인데, 튜플의 리스트로 담고 있는 형태로 바꿔줌.

rec.array([(  0, 1959., 1.,  2710.349, 1707.4,  286.898,  470.045,  1886.9,  28.98 ,  139.7,  2.82,  5.8, 177.146,  0.  ,  0.  ),
           (  1, 1959., 2.,  2778.801, 1733.7,  310.859,  481.301,  1919.7,  29.15 ,  141.7,  3.08,  5.1, 177.83 ,  2.34,  0.74),
           (  2, 1959., 3.,  2775.488, 1751.8,  289.226,  491.26 ,  1916.4,  29.35 ,  140.5,  3.82,  5.3, 178.657,  2.74,  1.09),
           (  3, 1959., 4.,  2785.204, 1753.7,  299.356,  484.052,  1931.3,  29.37 ,  140. ,  4.33,  5.6, 179.386,  0.27,  4.06),
           (  4, 1960., 1.,  2847.699, 1770.5,  331.722,  462.199,  1955.5,  29.54 ,  139.6,  3.5 ,  5.2, 180.007,  2.31,  1.19),
           (  5, 1960., 2.,  2834.39 , 1792.9,  298.152,  460.4  ,  1966.1,  29.55 ,  140.2,  2.68,  5.2, 180.671,  0.14,  2.55),
           (  6, 1960., 3.,  2839.022, 1785.8,  296.375,  474.676,  1967.8,  29.75 ,  140.9,  2.36,  5.6, 181.528,  2.7 , -0.34),
           (  7, 1960., 4.,  2802.616, 1788.2,  259.764,  476.434,  1966.6,  29.84 ,  141.

In [102]:
data = DataFrame(data.to_records(),
                 columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                 index=periods.to_timestamp('D', 'end'))
data        # 분기를 조정 가능함.     

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2
1960-06-30,2834.390,0.14,5.2
1960-09-30,2839.022,2.70,5.6
1960-12-31,2802.616,1.21,6.3
1961-03-31,2819.264,-0.40,6.8


In [ ]:
data.stack().reset_index().rename(columns={0: 'value'})

In [ ]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
wdata = ldata.pivot('date', 'item', 'value')

In [ ]:
ldata[:10]
wdata

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted.head()

In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

In [ ]:
pivoted['value'][:5]

In [ ]:
ldata.set_index(['date', 'item'])

In [ ]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

## Data transformation

### Removing duplicates

In [ ]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

In [ ]:
#data.drop_duplicates(['k1', 'k2'], take_last=True)
data.drop_duplicates(['k1', 'k2'], keep='last')

### Transforming data using a function or mapping

In [ ]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
str.lower(data['food'])

In [ ]:
%timeit data['food'].map(str.lower).map(meat_to_animal)

In [ ]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

In [ ]:
%timeit data['food'].map(lambda x: meat_to_animal[x.lower()])

### Replacing values

In [ ]:
data = Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
data[data == -999] = np.nan

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### Renaming axis indexes

In [ ]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

In [ ]:
data.index
data.index.map(str.upper)

In [ ]:
data.index = data.index.map(str.upper)
data

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

In [ ]:
data.rename(index={'OHIo': 'INDIANA', 'NEW YORK': 'NY'},
            columns={'three': 'peekaboo'})

In [ ]:
# Always returns a reference to a DataFrame
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretization and binning

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [ ]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

In [ ]:
cats.codes
#cats.labels

In [ ]:
cats.categories
#cats.levels

In [ ]:
pd.value_counts(cats)

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

In [ ]:
pd.value_counts(pd.cut(ages, bins, labels=group_names))

In [ ]:
data = np.random.rand(20)
pd.value_counts(pd.cut(data, 4, precision=2))
data

In [ ]:
catal = pd.cut(data, 5)
data
catal

In [ ]:
pd.value_counts(catal)

In [ ]:
data = np.random.randn(1000) # Normally distributed
cats = pd.qcut(data, 4) # Cut into quartiles
cats

In [ ]:
pd.value_counts(cats)

In [ ]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
pd.value_counts(pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]))

### Detecting and filtering outliers

In [ ]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000, 4))
data.head()
data.describe()

In [ ]:
col = data[3]
col[np.abs(col) > 3]

In [ ]:
np.abs(data) > 3
(np.abs(data) > 3).any(1)

In [ ]:
data[(np.abs(data) > 3).any(1)]

In [ ]:
np.abs(data) > 3

In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

In [ ]:
data.head(10)

### Permutation and random sampling

In [ ]:
df = DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

In [ ]:
df

In [ ]:
df.take(sampler)

In [ ]:
df.take(np.random.permutation(len(df))[:3])

In [ ]:
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=10)

In [ ]:
sampler

In [ ]:
draws = bag.take(sampler)
draws

In [ ]:
bag[sampler]

## 로또 시뮬레이션

In [ ]:
def show_me_the_lotto(money=10000, lotto_price=1000):
    return Series(np.arange(money/lotto_price))\
                .map(lambda x: np.sort(np.random.permutation(45)[:6] + 1))

In [ ]:
show_me_the_lotto()

### Computing indicator / dummy variables

In [ ]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
df
pd.get_dummies(df['key'])

In [ ]:
df[['data1']]

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

In [ ]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ch02/movielens/movies.dat', sep='::', header=None,
                        names=mnames)
movies[:10]

In [ ]:
genre_iter = (set(x.split('|')) for x in movies.genres)
genres = sorted(set.union(*genre_iter))
genres

In [ ]:
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)
dummies

In [ ]:
gen1 = movies.genres[0]; gen1
gen1.split('|')

In [ ]:
for i, gen in enumerate(movies.genres):
    dummies.loc[i, gen.split('|')] = 1

In [ ]:
dummies

In [ ]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.loc[0]

In [ ]:
np.random.seed(12345)

In [ ]:
values = np.random.rand(10)
values

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

## String manipulation

### String object methods

In [ ]:
val = 'a,b,  guido'
val.split(',')

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

In [ ]:
'::'.join(pieces)

In [ ]:
'guido' in val

In [ ]:
val.index(',')

In [ ]:
val.find(':')

In [ ]:
val.index(':')

In [ ]:
val.count(',')

In [ ]:
val.replace(',', '::')

In [ ]:
val.replace(',', '')

### Regular expressions

In [ ]:
import re
text = "foo    bar\t baz  \tqux"
print(text)
text.split(' ')
re.split('\s+', text)
re.findall('\w+', text)

In [ ]:
regex = re.compile('\s+')
regex.split(text)

In [ ]:
regex.findall(text)

In [ ]:
text = """Dave dave@google.com
Iceman iceman@snu.ac.kr
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
regex.findall(text)

In [ ]:
m = regex.search(text)
m
type(m)

In [ ]:
m.start()
m.end()
text[m.start():m.end()]

In [ ]:
print(regex.match(text))

In [ ]:
print(text)
print(regex.sub('REDACTED', text))

In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
m = regex.match('wesm@bright.abc.net.com')
m.groups()
m.group(1)
m.group(0)

In [ ]:
regex.findall(text)

In [ ]:
print(text)

In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

In [ ]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groupdict()

In [ ]:
m_dict = m.groupdict()
m_dict['username']

### Vectorized string functions in pandas

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)
data

In [ ]:
data.isnull()
data[data.notnull()]

In [ ]:
data.str.contains('gmail')

In [ ]:
pattern

In [ ]:
regex.findall(data)

In [ ]:
data[data.notnull()].map(regex.findall)

In [ ]:
df = DataFrame(data)
df

In [ ]:
df[:-1].applymap(regex.findall)

In [ ]:
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

In [ ]:
matches.str.get(1)

In [ ]:
matches.str[0]

In [ ]:
data.str[:5]

## Example: USDA Food Database

In [ ]:
import json
db = json.load(open('ch07/foods-2011-10-03.json'))
len(db)
type(db)

In [ ]:
db[0]

In [ ]:
db[0].keys()

In [ ]:
len(db[0]['nutrients'])
db[0]['nutrients'][0]

In [ ]:
nutrients = DataFrame(db[0]['nutrients'])
nutrients[:7]

In [ ]:
info_keys = ['description', 'group', 'id', 'manufacturer']
info = DataFrame(db, columns=info_keys)

In [ ]:
info[:5]

In [ ]:
info

In [ ]:
pd.value_counts(info.group)[:10]

In [ ]:
nutrients = []

for rec in db:
    fnuts = DataFrame(rec['nutrients'])
    fnuts['id'] = rec['id']
    nutrients.append(fnuts)

nutrients = pd.concat(nutrients, ignore_index=True)

In [ ]:
nutrients

In [ ]:
nutrients.duplicated().sum()

In [ ]:
nutrients = nutrients.drop_duplicates()

In [ ]:
col_mapping = {'description' : 'food',
               'group'       : 'fgroup'}
info = info.rename(columns=col_mapping, copy=False)
info

In [ ]:
col_mapping = {'description' : 'nutrient',
               'group' : 'nutgroup'}
nutrients = nutrients.rename(columns=col_mapping, copy=False)
nutrients

In [ ]:
ndata = pd.merge(nutrients, info, on='id', how='outer')

In [ ]:
ndata

In [ ]:
len(ndata)
ndata.loc[30000]

In [ ]:
len(ndata[ndata.nutrient == 'Glycine'])

In [ ]:
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)
#result['Zinc, Zn'].order().plot(kind='barh')
result['Zinc, Zn'].sort_values().plot(kind='barh')

In [ ]:
by_nutrient = ndata.groupby(['nutgroup', 'nutrient'])

get_maximum = lambda x: x.xs(x.value.idxmax())
get_minimum = lambda x: x.xs(x.value.idxmin())

max_foods = by_nutrient.apply(get_maximum)[['value', 'food']]

# make the food a little smaller
max_foods.food = max_foods.food.str[:50]

In [ ]:
max_foods.loc['Amino Acids']['food']